<a href="https://colab.research.google.com/github/vinay10949/AnalyticsAndML/blob/master/FeatureEngineering/PipeLine/1_Feature_Tools_Aggregating_transactional_data_with_math_operations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install featuretools

In [0]:
import pandas as pd
import featuretools as ft

In [0]:
# load data from feature tools

data_dict = ft.demo.load_mock_customer()

In [4]:
# each table in the dictionaty is a pandas dataframe

type(data_dict["transactions"])

pandas.core.frame.DataFrame

In [5]:
# capture all 3 data tables into a pandas dataframe

data = data_dict["transactions"].merge(
    data_dict["sessions"]).merge(data_dict["customers"])

data.shape

(500, 11)

In [0]:
# select the columns with which we will work in this recipe

cols = ['customer_id',
        'transaction_id',
        'transaction_time',
        'amount'
        ]

data = data[cols]

In [7]:
# display data

data.head()

,customer_id,transaction_id,transaction_time,amount
0,2,298,2014-01-01 00:00:00,127.64
1,2,2,2014-01-01 00:01:05,109.48
2,2,308,2014-01-01 00:02:10,95.06
3,2,116,2014-01-01 00:03:15,78.92
4,2,371,2014-01-01 00:04:20,31.54


In [8]:
print('Number of customers: {}'.format(data['customer_id'].nunique()))
print('Number of transactions: {}'.format(data['transaction_id'].nunique()))

Number of customers: 5
Number of transactions: 500


In [9]:
# dispay number of transactions per customer

data.groupby('customer_id')['transaction_id'].count()

customer_id
1    126
2     93
3     93
4    109
5     79
Name: transaction_id, dtype: int64

## Aggregation with pandas

In [11]:
# make a list of the operations we want to perform
operations = ['sum', 'max', 'min', 'mean', 'median', 'std', 'count']

# make a list of the new feature names
feature_names = [
    'total_amount', 'max_amount', 'min_amount', 'mean_amount', 'median_amount',
    'std_amount', 'number of transactions'
]

df = pd.DataFrame()

# capture new features in a pandas dataframe
df[feature_names] = data.groupby('customer_id')['amount'].agg(operations)

df

,total_amount,max_amount,min_amount,mean_amount,median_amount,std_amount,number of transactions
customer_id,,,,,,,
1,9025.62,139.43,5.81,71.631905,69.715,40.442059,126
2,7200.28,146.81,8.73,77.422366,75.960,37.705178,93
3,6236.62,149.15,5.89,67.060430,58.930,43.683296,93
4,8727.68,149.95,5.73,80.070459,81.410,45.068765,109
5,6349.66,149.02,7.55,80.375443,78.870,44.095630,79


## Aggregation with Featuretools

In [12]:
# in order for feature tools to work, we need to create
# entity sets

# create and entity set and give it a name
es = ft.EntitySet(id="customer_data")

# add transactions table to the entity set
# basically we pass the entire dataframe indicating
# the transaction_id as unique identifier
# and the transaction time as the time index
# there is one unique timestamp per transaction
es.entity_from_dataframe(entity_id='transactions',
                         dataframe=data[cols],
                         index="transaction_id",
                         time_index='transaction_time')

# display the entity set
es

Entityset: customer_data
  Entities:
    transactions [Rows: 500, Columns: 4]
  Relationships:
    No relationships

In [13]:
# now we indicate that within the entity set
# there is another table, with customer data,
# each customer identified with a unique
# customer id

es.normalize_entity(base_entity_id="transactions",
                    new_entity_id="customers",
                    index="customer_id")

Entityset: customer_data
  Entities:
    transactions [Rows: 500, Columns: 4]
    customers [Rows: 5, Columns: 2]
  Relationships:
    transactions.customer_id -> customers.customer_id

In [14]:
# let's print out the operations we want to perform
# we created this list in cell 9

operations

['sum', 'max', 'min', 'mean', 'median', 'std', 'count']

In [15]:
# set up the feature creation object
# to aggregate transactions at customer level

feature_matrix, features = ft.dfs(entityset=es,
                                  target_entity="customers",
                                  agg_primitives=operations,
                                  trans_primitives=[],
                                  verbose=True,
                                  )

# display name of created features
features

Built 7 features
Elapsed: 00:00 | Remaining: 00:00 | Progress: 100%|██████████| Calculated: 1/1 chunks


[<Feature: SUM(transactions.amount)>,
 <Feature: MAX(transactions.amount)>,
 <Feature: MIN(transactions.amount)>,
 <Feature: MEAN(transactions.amount)>,
 <Feature: MEDIAN(transactions.amount)>,
 <Feature: STD(transactions.amount)>,
 <Feature: COUNT(transactions)>]

In [16]:
# dataframe with the new features
feature_matrix

,SUM(transactions.amount),MAX(transactions.amount),MIN(transactions.amount),MEAN(transactions.amount),MEDIAN(transactions.amount),STD(transactions.amount),COUNT(transactions)
customer_id,,,,,,,
1,9025.62,139.43,5.81,71.631905,69.715,40.442059,126
2,7200.28,146.81,8.73,77.422366,75.960,37.705178,93
3,6236.62,149.15,5.89,67.060430,58.930,43.683296,93
4,8727.68,149.95,5.73,80.070459,81.410,45.068765,109
5,6349.66,149.02,7.55,80.375443,78.870,44.095630,79


If you compare this table to the one created in 8, the values should be identical.